#12. <span style="color:darkblue; font-family:Courier">BuildAction</span> and <span style="color:darkblue; font-family:Courier">BuildCheck</span>

This is a somewhat advanced topic. In some cases, it is desirable to trigger actions to be done as part of the model building process. The <span style="color:darkblue; font-family:Courier">BuildAction</span> function provides this capability in a Pyomo model. It takes as arguments optional index sets and a function to peform the action. For example,

In [2]:
model.BuildBpts = BuildAction(model.J, rule=bpts_build)

NameError: name 'BuildAction' is not defined

calls the function <span style="color:darkblue; font-family:Courier">bpts_build</span> for each member of <span style="color:darkblue; font-family:Courier">model.J</span>. The function <span style="color:darkblue; font-family:Courier">bpts_build</span> should have the model and a variable for the members of <span style="color:darkblue; font-family:Courier">model.J</span> as formal arguments. In this example, the following would be a valid declaration for the function:

In [3]:
def bpts_build(model, j):

SyntaxError: unexpected EOF while parsing (<ipython-input-3-21def2ea90d9>, line 1)

A full example, which extends the [abstract2.py](https://software.sandia.gov/downloads/pub/pyomo/PyomoOnlineDocs.html#abstract2.py) and [abstract2piece.py](https://software.sandia.gov/downloads/pub/pyomo/PyomoOnlineDocs.html#abstract2piece.py) examples, is

In [4]:
# abstract2piecebuild.py
# Similar to abstract2piece.py, but the breakpoints are created using a build action

from pyomo.environ import *

model = AbstractModel()

model.I = Set()
model.J = Set()

model.a = Param(model.I, model.J)
model.b = Param(model.I)
model.c = Param(model.J)

model.Topx = Param(default=6.1) # range of x variables
model.PieceCnt = Param(default=100)

# the next line declares a variable indexed by the set J
model.x = Var(model.J, domain=NonNegativeReals, bounds=(0,model.Topx))
model.y = Var(model.J, domain=NonNegativeReals)

# to avoid warnings, we set breakpoints beyond the bounds
# we are using a dictionary so that we can have different
# breakpoints for each index. But we won't.
model.bpts = {}
def bpts_build(model, j):
    model.bpts[j] = []
    for i in range(model.PieceCnt+2):
        model.bpts[j].append(float((i*model.Topx)/model.PieceCnt))
# The object model.BuildBpts is not refered to again;
# the only goal is to trigger the action at build time
model.BuildBpts = BuildAction(model.J, rule=bpts_build)

def f4(model, j, xp):
    # we not need j in this example, but it is passed as the index for the constraint
    return xp**4

model.ComputePieces = Piecewise(model.J, model.y, model.x, pw_pts=model.bpts, pw_constr_type='EQ', f_rule=f4)

def obj_expression(model):
    return summation(model.c, model.y)

model.OBJ = Objective(rule=obj_expression)

def ax_constraint_rule(model, i):
    # return the expression for the constraint for i
    return sum(model.a[i,j] * model.x[j] for j in model.J) >= model.b[i]

# the next line creates one constraint for each member of the set model.I
model.AxbConstraint = Constraint(model.I, rule=ax_constraint_rule)

This example uses the build action to create a model component with breakpoints for a [piecewise](https://software.sandia.gov/downloads/pub/pyomo/PyomoOnlineDocs.html#piecewise) function. The <span style="color:darkblue; font-family:Courier">BuildAction</span> is triggered by the assignment to <span style="color:darkblue; font-family:Courier">model.BuildBpts</span>. This object is not referenced again, the only goal is to cause the execution of <span style="color:darkblue; font-family:Courier">bpts_build</span>, which places data in the <span style="color:darkblue; font-family:Courier">model.bpts</span> dictionary. Note that if <span style="color:darkblue; font-family:Courier">model.bpts</span> had been a <span style="color:darkblue; font-family:Courier">Set</span>, then it could have been created with an initialize argument to the <span style="color:darkblue; font-family:Courier">Set</span> declaration. Since it is a special-purpose dictionary to support the [piecewise](https://software.sandia.gov/downloads/pub/pyomo/PyomoOnlineDocs.html#piecewise) functionality in Pyomo, we use a <span style="color:darkblue; font-family:Courier">BuildAction</span>.

Another application of <span style="color:darkblue; font-family:Courier">BuildAction</span> can be intialization of Pyomo model data from Python data structures, or efficient initialization of Pyomo model data from other Pyomo model data. Consider the [Isinglecomm.py](https://software.sandia.gov/downloads/pub/pyomo/PyomoOnlineDocs.html#Isinglecomm.py) example. Rather than using an initialization for each list of sets <span style="color:darkblue; font-family:Courier">NodesIn</span> and <span style="color:darkblue; font-family:Courier">NodesOut</span> separately using <span style="color:darkblue; font-family:Courier">initialize</span>, it is a little more efficient and probably a little clearer, to use a build action.

The full model is :

In [6]:
# Isinglecomm.py
# NodesIn and NodesOut are created by a build action using the Arcs
from pyomo.environ import *

model = AbstractModel()

model.Nodes = Set()
model.Arcs = Set(dimen=2)

model.NodesOut = Set(model.Nodes, within=model.Nodes, initialize=[])
model.NodesIn = Set(model.Nodes, within=model.Nodes, initialize=[])

def Populate_In_and_Out(model):
    # loop over the arcs and put the end points in the appropriate places
    for (i,j) in model.Arcs:
        model.NodesIn[j].add(i)
        model.NodesOut[i].add(j)

model.In_n_Out = BuildAction(rule = Populate_In_and_Out)

model.Flow = Var(model.Arcs, domain=NonNegativeReals)
model.FlowCost = Param(model.Arcs)

model.Demand = Param(model.Nodes)
model.Supply = Param(model.Nodes)

def Obj_rule(model):
    return summation(model.FlowCost, model.Flow)
model.Obj = Objective(rule=Obj_rule, sense=minimize)

def FlowBalance_rule(model, node):
    return model.Supply[node] \
     + sum(model.Flow[i, node] for i in model.NodesIn[node]) \
     - model.Demand[node] \
     - sum(model.Flow[node, j] for j in model.NodesOut[node]) \
     == 0
model.FlowBalance = Constraint(model.Nodes, rule=FlowBalance_rule)

for this model, the same data file can be used as for [Isinglecomm.py](https://software.sandia.gov/downloads/pub/pyomo/PyomoOnlineDocs.html#Isinglecomm.py) such as the toy data file:

In [7]:
# Isinglecomm.dat: data for Isinglecomm.py

set Nodes := CityA CityB CityC ;

set Arcs :=
CityA CityB
CityA CityC
CityC CityB
;

param : FlowCost :=
CityA CityB 1.4
CityA CityC 2.7
CityC CityB 1.6
 ;

param Demand :=
CityA 0
CityB 1
CityC 1
;

param Supply :=
CityA 2
CityB 0
CityC 0
;

SyntaxError: invalid syntax (<ipython-input-7-3f129f2ffb10>, line 3)

Build actions can also be a way to implement data validation, particularly when multiple Sets or Parameters must be analyzed. However, the the <span style="color:darkblue; font-family:Courier">BuildCheck</span> component is prefered for this purpose. It executes its rule just like a <span style="color:darkblue; font-family:Courier">BuildAction</span> but will terminate the construction of the model instance if the rule returns <span style="color:darkblue; font-family:Courier">False</span>.